# FEM Solution of 1-D transient heat transfer in a rod with composite material and internal heat generation

![](Figure.jpg)



- **Ambient temperature** (Constant, Constant rate, One hold temperature cycle)
        
- **Boundary Condition** (Convection heat transfer)
        
- **Solutiion method** (Explicit Euler, Implicit Euler w/ Newton Raphson)
            
- **Material** (AS4/3501-6)
            
- **Mesh** (Uniform)
            
- **Element** (2-Node Linear, 3-Node Nonlinear)

In [1]:
# import
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
import scipy as sp
from scipy import linalg
from ipywidgets import interactive
from IPython.display import clear_output
%matplotlib inline

In [2]:
#set figure defaults for IPython notebook (must be in separate cell from above)
matplotlib.rcParams.update({'font.size': 18, 'lines.linewidth':4})

## Main functions

In [3]:
# Functions 
# Air temperature function, Constant, Constant Rate, OneHold
def T_air(t,typ,T_start,T_hold,T_const,T_rate,th1,th2):
    # air_temp_type, 'Constant', 'Constant_rate', 'OneHold'
    # T_start, start air temperature (K)
    # T_const, air consat temperate (for 'Constat' type) (K)
    # T_rate, air temperature increase rate (for 'Constant_rate' type)
    # T_hold, air hold temperate (for 'OneHold' type) (K)
    # th1, time for start of hold (for 'OneHold' type) (seconds)
    # th2, time for end of hold (for 'OneHold' type) (seconds)
    if typ == 'ConstantRate':
        T_a = t*T_rate + T_start
    elif typ == 'OneHold':
        T_rate = (T_hold-T_start) / th1
        if t < th1:
            T_a = t*T_rate + T_start
        elif t > th2:
            T_a = -(t-th2)*T_rate + T_hold
        else:
            T_a = T_hold
    elif typ == 'Constant':
        T_a = T_const
    return T_a


# Material model
# C Coefficients
def C(Temp):
    # Table 5.2 of S. Amini Niaki thesis for 3501-6
    A1 = 3.5017e7
    A2 = -3.3567e7
    A3 = 3.2667e3
    A = [A1, A2, A3]
    dE1 = 80700
    dE2 = 77800
    dE3 = 56600
    dE = [dE1, dE2, dE3]
    R = 8.314
    C = list(range(3))
    for i in range(3):
        C[i]= A[i] * np.exp(-dE[i]/(R*Temp))
    return C


# Rate of degree of cure function 
def alpha_dot_func(alpha,T):  
    # Table 5.2 of S.A. Niaki thesis for 3501-6  
    B = 0.47
    if alpha <= 0.3:
        alpha_dot = (C(T)[0]+(C(T)[1]*alpha)) * (1-alpha) * (B-alpha)
    else:
        alpha_dot = C(T)[2]*(1-alpha) 
    return alpha_dot


# Degree of cure
def alpha_func(el,t_start,t_end,delt,T_sol):
    alpha = np.array([0])
    for t in np.arange(t_start+delt,t_end,delt):
        if Element == 'Linear':
            T_el = 0.5*(T_sol[el-1,int((t-t_start)/delt)]+T_sol[el,int((t-t_start)/delt)])
        elif Element == 'Nonlinear':
            T_el = (T_sol[2*el-1,int((t-t_start)/delt)]) # center node
            
        alpha_dot = alpha_dot_func(alpha[-1],T_el)
        alpha = np.append(alpha, alpha[-1] + delt * alpha_dot)
    return alpha


# Arrays carrying information about the mesh, 2 node elements
def Mesh(Coords_start,Length,num_el):    
    Nodes = np.linspace(1,num_el+1,num_el+1)
    Elements= np.zeros((num_el,2))
    Elements[:,0] = np.linspace(1,num_el,num_el)
    Elements[:,1] = np.linspace(2,num_el+1,num_el)
    Coords = np.linspace(Coords_start,Coords_start+Length,num_el+1)
    return Nodes, Elements, Coords

# Arrays carrying information about the mesh, 3 node elements
def Mesh3(Coords_start,Length,num_el):    
    Nodes = np.linspace(1,2*num_el+1,2*num_el+1)
    Elements= np.zeros((num_el,3))
    Elements[:,0] = np.linspace(1,2*num_el-1,num_el)
    Elements[:,1] = np.linspace(2,2*num_el,num_el)
    Elements[:,2] = np.linspace(3,2*num_el+1,num_el)
    Coords = np.linspace(Coords_start,Coords_start+Length,2*num_el+1)
    return Nodes, Elements, Coords

# Element K, C, C_lumped, and F matrixes, 2 node element
def KCF(el,a,b,alpha_dot_el,Coords):    
    L = Coords[el] - Coords[el-1] # element length    
    K_el = np.zeros((2,2))
    C_el = np.zeros((2,2))
    F_el = np.zeros(2)
    for kesi in [-1/(3**0.5), 1/(3**0.5)]:  # 2 gauss points      
        # shpae functions
        N = np.array([(1-kesi)/2, (1+kesi)/2])
        B = 0.5 * np.array([-1, 1])        
        K_el += (2/L) * a * np.outer(B,B)
        C_el += (L/2) * np.outer(N,N)
        F_el += (L/2) * alpha_dot_el * b * N
    C_lumped_el = L/2 * np.array([[1, 0],[0, 1]]) 
    return K_el, C_el, C_lumped_el, F_el

# Element K, C, C_lumped, and F matrixes, 3 node element
def KCF3(el,a,b,alpha_dot_el,Coords):    
    L = Coords[2*el] - Coords[2*el-2] # element length    
    K_el = np.zeros((3,3))
    C_el = np.zeros((3,3))
    F_el = np.zeros(3)
    i = 0
    weight = [5/9, 8/9, 5/9]
    for kesi in [-((3/5)**0.5), 0, ((3/5)**0.5)]:  # 3 gauss points      
        # shpae functions
        N = np.array([kesi*(kesi-1)/2, 1-kesi**2, kesi*(kesi+1)/2]) * weight[i]
        B = np.array([kesi-0.5, -2*kesi, kesi+0.5])* weight[i]        
        K_el += (2/L) * a * np.outer(B,B)
        C_el += (L/2) * np.outer(N,N)
        F_el += (L/2) * alpha_dot_el * b * N
        i += 1
    C_lumped_el = L/2 * np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]]) 
    return K_el, C_el, C_lumped_el, F_el

# Assembly of element array into global arrays, 2 node elements
def Assemble(el,K_el,C_el,C_lumped_el,F_el,K_sys,C_sys,C_lumped_sys,F_sys):    
    ii = el-1
    for i in range(2):
        F_sys[ii] += F_el[i]
        jj = el-1
        for j in range(2):
            K_sys[ii][jj] += K_el[i][j]
            C_sys[ii][jj] += C_el[i][j]
            C_lumped_sys[ii][jj] += C_lumped_el[i][j]
            jj += 1
        ii += 1 
    return K_sys, C_sys, C_lumped_sys, F_sys

# Assembly of element array into global arrays, 3 node elements
def Assemble3(el,K_el,C_el,C_lumped_el,F_el,K_sys,C_sys,C_lumped_sys,F_sys):    
    ii = 2*el-2
    for i in range(3):
        F_sys[ii] += F_el[i]
        jj = 2*el-2
        for j in range(3):
            K_sys[ii][jj] += K_el[i][j]
            C_sys[ii][jj] += C_el[i][j]
            C_lumped_sys[ii][jj] += C_lumped_el[i][j]
            jj += 1
        ii += 1 
    return K_sys, C_sys, C_lumped_sys, F_sys


## Visualization functions

In [4]:

# Temperature plots
def plot_Temp(t,t_start,delt,T_ini,Analysis):
    # Temperature distribution along the rod at specific instance of t
    plt.figure(figsize=(10,10))   
    plt.subplot(211)
    plt.scatter(Coords,T_sol[:,int((t-t_start)/delt)])
    plt.plot(Coords,T_sol[:,int((t-t_start)/delt)])
    plt.xlabel('x location (m)')
    plt.ylabel('Temperature (K)')
    plt.ylim(round(T_ini)-1,round(np.amax(T_sol))*1.1)
    plt.title('delt = {} s,  Analysis : {}, Element: {}, Heat generation: {}\n a = {}, b={}, Ch ={}'.
              format(delt,Analysis,Element,heat_gen,a,b,Ch), fontsize=14, loc="left")
    plt.show()
    
    # Air temperature evolution
    plt.figure(figsize=(10,10))
    plt.subplot(212)
    T_air_arr = np.array([T_air(0,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2)])
    for time in np.arange(t_start+delt,t_end,delt):
        T_air_arr = np.append(T_air_arr, T_air(time,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2))         
    plt.plot(np.arange(t_start,t_end,delt),T_air_arr)
    plt.plot([t,t],[T_ini,round(np.amax(T_air_arr))*1.1], color = 'red')
    plt.xlabel('Time (s)')
    plt.ylabel('Air temperature (K)')
    plt.ylim(round(T_ini)-1,round(np.amax(T_air_arr))*1.1)
    plt.show()

    
# Elements degree of cure plots
def plot_alpha(el,t,t_start,t_end,delt,T_sol):    
    # Degree of cure for element el
    alpha_arr = alpha_func(el,t_start,t_end,delt,T_sol)
    plt.figure(figsize=(10,15))  
    plt.subplot(311)
    plt.plot(np.arange(t_start,t_end,delt),alpha_arr)
    plt.xlabel('Time (s)')
    plt.ylabel('Element degree of cure')
    plt.ylim(0,np.amax(alpha_arr)*1.1)
    plt.show()
    # Rate of degree of cure for element el
    alpha_dot_arr = np.array([])
    for i in range(np.size(alpha_arr)):
        alpha = alpha_arr[i]
        if Element == 'Linear':
            T = 0.5*(T_sol[el-1,i]+T_sol[el,i])
        elif Element == "Nonlinear":    
            T = T_sol[2*el-1,i] # Center node
        alpha_dot = alpha_dot_func(alpha,T)
        alpha_dot_arr = np.append(alpha_dot_arr, alpha_dot)        
    plt.figure(figsize=(10,15))  
    plt.subplot(312)
    plt.plot(np.arange(t_start,t_end,delt),alpha_dot_arr)
    plt.xlabel('Time (s)')
    plt.ylabel('Element rate of degree of cure')
    plt.ylim(0,np.amax(alpha_dot_arr)*1.1)
    plt.show()
    
    # Element el temperature evolution
    plt.figure(figsize=(10,15))
    plt.subplot(313)
    
    if Element == 'Linear':
        T_el_arr = 0.5*(T_sol[el-1,:]+T_sol[el,:])
    elif Element == "Nonlinear":
        T_el_arr = (T_sol[2*el-1,:]) # Center node
        
    plt.plot(np.arange(t_start,t_end,delt),T_el_arr, label="element")
    T_air_arr = np.array([T_air(0,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2)])
    for time in np.arange(t_start+delt,t_end,delt):
        T_air_arr = np.append(T_air_arr, T_air(time,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2))         
    plt.plot(np.arange(t_start,t_end,delt),T_air_arr, label="air")
    plt.legend(loc='best')
    plt.xlabel('Time(s)')
    plt.ylabel('Temperature (K)')
    plt.show()

## Inputs

In [5]:

# material properties
rho_c = 1463 # composites density (kg/m3) 
# --> 1463 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
#--> 1790 for AS4 carbon (https://www.900gpa.com/en/product/fiber/CF_001EF245BC?u=metric)
k_c = 0.65 # composites thermal conductivity (W/m K) 
# --> 0.65 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
#--> 6.83 for AS4 carbon (https://www.900gpa.com/en/product/fiber/CF_001EF245BC?u=metric)
Cp_c = 1200 # composite specific heat capacity (J/kg K) 
# --> 1200 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
# --> 1300 for AS4 Carbon (https://www.researchgate.net/figure/Specific-heat-capacity-of-AS4-carbon-fiber-PES-matrix-and-CF-PES-tape-fiber-volume_fig6_320801788)
rho_r = 1256 # resin density  (kg/m3), 
# -->1256 for 3501-6 (https://www.researchgate.net/figure/3-Properties-of-Hexcel-3501-6-Epoxy-Resin-17_tbl3_267585693)
H_r = 400e3 # resin heat of reasction per unit mass (J / kg)
# --> 400*1000  for 3501-6 (https://books.google.ca/books?id=p__RBQAAQBAJ&pg=PA478&lpg=PA478&dq=resin+3501-6+heat+reaction+per+unit+mass&source=bl&ots=yzGE-Cu-Fo&sig=ACfU3U07FEurjhNeAVzwOKofNp-Y_zYDdw&hl=en&sa=X&ved=2ahUKEwjut6Lx2OboAhUMrp4KHf90BkAQ6AEwAHoECAsQLA#v=onepage&q=resin%203501-6%20heat%20reaction%20per%20unit%20mass&f=false)
nu_r = 0.33 # resin volume fraction in composite material 
# --> 0.33
h = 120; # convection heat trasnfer coefficient (W/ m2 K)
# --> 120 in autoclave (https://www.semanticscholar.org/paper/HEAT-TRANSFER-COEFFICIENT-DISTRIBUTION-INSIDE-AN-Slesinger-Shimizu/b61dfa6b4811edb51b003e43cc61088f0d13e348)


a =  1#k_c/(rho_c*Cp_c)
b =  1#rho_r*H_r*nu_r/(rho_c*Cp_c)
Ch = 1#h/k_c; 

t_start = 0 # start time (seconds)
t_end = 230*60 # end time (seconds)
delt = 0.01 # time step (seconds)

# geometry
Length = 100 # rod length (m)
num_el = 20 # number of elements
Coords_start = 0 # first node x coordinate

# air temperautre
air_temp_type = 'OneHold' # 'Constant', 'ConstantRate', 'OneHold'
T_start = 20+273 # start air temperature (K)
T_const = 180+273 # air consat temperate (for 'Constat' type) (K)
T_rate = 0.5 # air temperature increase rate (for 'Constant_rate' type)
T_hold = 170+273 # air hold temperate (for 'OneHold' type) (K)
th1 = 70*60 # time for start of hold (for 'OneHold' type) (seconds)
th2 = 170*60 # time for end of hold (for 'OneHold' type) (seconds)

T_ini = T_air(0,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2) # initital temperature of material

# analysis type
Analysis = 'Backward';  # 'Backward' or 'Forward', Backward is Implicit Euler w/ Newton Raphson, Forward is Expicit Euler
cri = 0.01 # convergence criteria value for Implicit analysis
Element = 'Linear' # 'Linear' or 'Nonlinear'

# heat generation switch
heat_gen = 'Yes' # 'Yes' or 'No'


## FE Solution

In [ ]:


if Element == 'Linear':
    Nodes, Elements, Coords = Mesh(Coords_start,Length,num_el)
    T_sys = np.full((num_el+1,1), T_ini)       
    T_sol = np.reshape(T_sys,(num_el+1,1))
    T_sys_new = T_sys
elif Element == "Nonlinear":
    Nodes, Elements, Coords = Mesh3(Coords_start,Length,num_el)
    T_sys = np.full((2*num_el+1,1), T_ini)       
    T_sol = np.reshape(T_sys,(2*num_el+1,1))
    T_sys_new = T_sys


alpha = np.zeros(num_el)
alpha_dot = np.zeros(num_el)

for t in np.arange(t_start+delt,t_end+delt,delt): # loop for each time step
    
    for el in range(1,num_el+1):
        if heat_gen == 'Yes':
            T_el = 0.5*(T_sys[el-1] + T_sys[el])
            alpha[el-1] += delt * alpha_dot[el-1]
            alpha_dot[el-1] = alpha_dot_func(alpha[el-1],T_el)
        else:
            alpha_dot[el-1] = 0
             
    norm = 1000
    # loop for each iteration (meaningful for implicit only)
    while norm>cri:
        if Element == 'Linear':
            K_sys = np.zeros([num_el+1,num_el+1])
            C_sys = np.zeros([num_el+1,num_el+1])
            C_lumped_sys = np.zeros([num_el+1,num_el+1])
            F_sys = np.zeros(num_el+1)
        elif Element == "Nonlinear":
            K_sys = np.zeros([2*num_el+1,2*num_el+1])
            C_sys = np.zeros([2*num_el+1,2*num_el+1])
            C_lumped_sys = np.zeros([2*num_el+1,2*num_el+1])
            F_sys = np.zeros(2*num_el+1)
    
        # creating global matrices from elements matrices
        for el in range(1,num_el+1):
            if Element == 'Linear':
                # Element arrays
                K_el, C_el, C_lumped_el, F_el = KCF(el,a,b,alpha_dot[el-1],Coords)
                # Assembly
                K_sys, C_sys, C_lumped_sys, F_sys = Assemble(el,K_el,C_el,C_lumped_el,F_el,K_sys,C_sys,C_lumped_sys,F_sys)
            elif Element == "Nonlinear": 
                # Element arrays
                K_el, C_el, C_lumped_el, F_el = KCF3(el,a,b,alpha_dot[el-1],Coords)
                # Assembly
                K_sys, C_sys, C_lumped_sys, F_sys = Assemble3(el,K_el,C_el,C_lumped_el,F_el,K_sys,C_sys,C_lumped_sys,F_sys)
            

        # Boundary condition
        T_a = T_air(t,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2) # current air temperature
        F_sys[0] += Ch * (T_a - T_sys[0])
        if Element == 'Linear':
            F_sys[num_el] += Ch * (T_a - T_sys[num_el])
            F_sys.shape = (num_el+1,1)
        elif Element == "Nonlinear": 
            F_sys[2*num_el] += Ch * (T_a - T_sys[2*num_el])
            F_sys.shape = (2*num_el+1,1)
        
        # Solution
        if Analysis == 'Forward':
            T_dot = sp.linalg.solve(C_lumped_sys, F_sys - K_sys.dot(T_sys)) 
            T_sys_new = T_sys + delt * T_dot # updating results
            norm = 0
        elif Analysis == 'Backward':
            # See S. Amini Niaki thesis, Equations A-6 to A-13
            LHS = C_sys + delt * K_sys
            RHS = delt * F_sys + C_sys.dot(T_sys)
            T_sys_old = T_sys_new
            T_sys_new = sp.linalg.solve(LHS, RHS)  # updating results
            norm = np.sum(np.square(T_sys_new-T_sys_old)) 
    clear_output(wait=True)
    print ("progress is : {}%".format(round((t-t_start)/(t_end-t_start)*100,1)))    
    T_sys = T_sys_new
    T_sol = np.append(T_sol, T_sys, axis=1)
            

progress is : 10.1%


## Output plots

In [10]:
interactive(lambda t=0: plot_Temp(t,t_start,delt,T_ini,Analysis), t=(t_start,t_end,(t_end-t_start)/10))

interactive(children=(FloatSlider(value=0.0, description='t', max=13800.0, step=1380.0), Output()), _dom_class…

In [11]:
interactive(lambda element=1: plot_alpha(element,t,t_start,t_end+delt,delt,T_sol), element=range(1,num_el+1))

interactive(children=(Dropdown(description='element', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, …